In [24]:
import MetaTrader5 as mt5
import schedule
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from  backtest import Backtester,  get_ohlc_history, create_price_fig, evaluate_backtest

In [25]:
# Conecte mt5
mt5.initialize()
# logine mt5
login = 52584692
password = 'apD2lh!ZGwmr1P'
server = 'ICMarketsSC-Demo'

mt5.login(login,password,server)

True

In [26]:
# this function retreives olhc data from mt5 account and return a data frame
def get_ohlc(symbol, timeframe, start_datetime, end_datetime):
    ohlc = mt5.copy_rates_range(symbol, timeframe, start_datetime, end_datetime)
    df = pd.DataFrame(ohlc)
    df['time'] = pd.to_datetime(df['time'], unit= 's')#.dt.tz_localize('UTC')
    return df

In [27]:
# 1 minut df
symbol = 'EURUSD'
pos_size = 1
timeframe = mt5.TIMEFRAME_M1
start_datetime = datetime(2025,9,1)
end_datetime = datetime.now()
df = get_ohlc(symbol, timeframe, start_datetime, end_datetime)
df

,time,open,high,low,close,tick_volume,spread,real_volume
0,2025-09-01 00:00:00,1.16943,1.16943,1.16908,1.16937,8,40,0
1,2025-09-01 00:01:00,1.16937,1.16938,1.16936,1.16937,5,54,0
2,2025-09-01 00:02:00,1.16937,1.16938,1.16935,1.16938,3,55,0
3,2025-09-01 00:03:00,1.16938,1.16938,1.16935,1.16938,5,55,0
4,2025-09-01 00:04:00,1.16938,1.16938,1.16935,1.16938,9,55,0
...,...,...,...,...,...,...,...,...
71921,2025-11-07 23:52:00,1.15608,1.15636,1.15606,1.15633,17,0,0
71922,2025-11-07 23:53:00,1.15634,1.15639,1.15633,1.15637,13,0,0
71923,2025-11-07 23:54:00,1.15637,1.15642,1.15637,1.15641,13,0,0
71924,2025-11-07 23:55:00,1.15638,1.15643,1.15633,1.15641,33,0,0


In [28]:
def lets_go():
    # 1 minut df
    symbol = 'EURUSD'
    timeframe = mt5.TIMEFRAME_M1
    start_datetime = datetime.now() - timedelta(days=30)
    end_datetime = datetime.now()
    df = get_ohlc(symbol, timeframe, start_datetime, end_datetime)
    # ensure mt5 connection alive
    if not mt5.initialize():
        print("Initialize failed:", mt5.last_error())
        return
    if not mt5.symbol_select(symbol, True):
        print(f"Failed to select symbol {symbol}")
        return
    
    df["atr"] = df["high"].rolling(14).max() - df["low"].rolling(14).min()
    df["vwap"] = (df["close"] * df["tick_volume"]).cumsum() / df["tick_volume"].cumsum()
    df["volume_spike"] = df["tick_volume"] > df["tick_volume"].rolling(10).mean() * 1.5
    # === Signal Generation ===
    def generate_signals(df):
        signals = []
        for i in range(14, len(df)):
            row = df.iloc[i]
            prev = df.iloc[i-1]
            cond1 = row["close"] < row["vwap"] - 2 * row["atr"]
            cond2 = row["spread"] < 0.0002
            cond3 = row["volume_spike"]
            cond4 = row["close"] > row["open"] and prev["close"] < prev["open"]

            if cond1 and cond2 and cond3 and cond4: 
                signals.append(i)
        return signals

    signals = generate_signals(df)

    # === Backtest ===

    results = []
    rr = 3  # risk-reward ratio

    for i in signals:
        entry_price = df.iloc[i]["close"]
        sl = entry_price - 0.5 * df.iloc[i]["atr"]
        tp = entry_price + rr * (entry_price - sl)

        for j in range(i+1, min(i+60, len(df))):  # look ahead for next ~2 hours
            high = df.iloc[j]["high"]
            low = df.iloc[j]["low"]
            if low <= sl:
                results.append(("loss", entry_price, sl))
                break
            elif high >= tp:
                results.append(("win", entry_price, tp))
                break
        else:
            # time-based exit
            close_exit = df.iloc[i+60]["close"]
            if close_exit > entry_price:
                results.append(("win", entry_price, close_exit))
            else:
                results.append(("loss", entry_price, close_exit))

    entry_price = df.iloc[i]["close"]
    df['sl'] = entry_price - 0.5 * df.iloc[i]["atr"]
    df['tp'] = entry_price + rr * (entry_price - sl)
    prev = df.iloc[i-1]
    def entring_stoploss_profit(row) :
        hour = row['time'].hour
        #idx = row.name
        #if idx == 0:
        #    return None
        #prev = df.iloc[idx - 1]

        if (
            row["close"] < row["vwap"] - 2 * row["atr"]
            and row["spread"] < 0.0002
            and row["volume_spike"]
            and row["close"] > row["open"]
            and prev["close"] < prev["open"]
            #and (hour < 11 or hour > 15)
        ):
            return "buy"
        return None
    df["signal"] = df.apply(entring_stoploss_profit, axis=1)
    tick = mt5.symbol_info_tick(symbol)
    lot = 0.01
    price = tick.ask
    buy_order_type = mt5.ORDER_TYPE_BUY
    buy_positions = mt5.positions_get(symbol=symbol)
    
    if df["signal"].iloc[-1] == "buy" and not len(buy_positions) :
        """if df["low"].iloc[-1] <= df["sl"].iloc[-1] :
            buy_sl = df["close"].iloc[-1]
        else :
            buy_sl = None
        if df["high"].iloc[-1] >= df["tp"].iloc[-1]:
            buy_tp = df["close"].iloc[-1]
        else :
            buy_tp = None"""
        
        request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": lot,
        "type": buy_order_type,
        "price": price,
        "sl": df["sl"],#buy_sl,
        "tp": df["tp"],#buy_tp,
        "deviation": 20,
        "magic": 234000,
        "comment": "buy executed",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
        }
        result = mt5.order_send(request)
        print("✅ Trade Request:", request)
        if result is None:
            print("order_send failed: no result returned")
            print("Error code:", mt5.last_error())
            return
        
        if result.retcode != mt5.TRADE_RETCODE_DONE:
            print(f"Order failed: {result.retcode}")
        else:
            print(f"buy trade placed successfully at {price} at {df["time"].iloc[-1]}")
            print(f"SL: {df["sl"]}")
    else:
        print(f"No trade condition met → skipping at {df["time"].iloc[-1]}")


In [ ]:
import schedule
import time
schedule.every(10).seconds.do(lets_go)

print("------> Strategy scheduler is running every 60 seconds...")
while True:
    schedule.run_pending()
    time.sleep(1)

------> Strategy scheduler is running every 60 seconds...
No trade condition met → skipping at 2025-11-07 23:56:00
No trade condition met → skipping at 2025-11-07 23:56:00


In [ ]:
#df["spread"] = df["spread"] / 1000
df["atr"] = df["high"].rolling(14).max() - df["low"].rolling(14).min()
df["vwap"] = (df["close"] * df["tick_volume"]).cumsum() / df["tick_volume"].cumsum()
df["volume_spike"] = df["tick_volume"] > df["tick_volume"].rolling(10).mean() * 1.5

# === Signal Generation ===
def generate_signals(df):
    signals = []
    for i in range(14, len(df)):
        row = df.iloc[i]
        prev = df.iloc[i-1]

        cond1 = row["close"] < row["vwap"] - 2 * row["atr"]
        cond2 = row["spread"] < 0.0002
        cond3 = row["volume_spike"]
        cond4 = row["close"] > row["open"] and prev["close"] < prev["open"]

        if cond1 and cond2 and cond3 and cond4: 
            signals.append(i)
    return signals

signals = generate_signals(df)

# === Backtest ===

results = []
rr = 3  # risk-reward ratio

for i in signals:
    entry_price = df.iloc[i]["close"]
    sl = entry_price - 0.5 * df.iloc[i]["atr"]
    tp = entry_price + rr * (entry_price - sl)

    for j in range(i+1, min(i+60, len(df))):  # look ahead for next ~2 hours
        high = df.iloc[j]["high"]
        low = df.iloc[j]["low"]
        if low <= sl:
            results.append(("loss", entry_price, sl))
            break
        elif high >= tp:
            results.append(("win", entry_price, tp))
            break
    else:
        # time-based exit
        close_exit = df.iloc[i+60]["close"]
        if close_exit > entry_price:
            results.append(("win", entry_price, close_exit))
        else:
            results.append(("loss", entry_price, close_exit))
# === Metrics ===
wins = [r for r in results if r[0] == "win"]
losses = [r for r in results if r[0] == "loss"]
win_rate = len(wins) / len(results)
r_r_ratio = np.mean([abs(w[2]-w[1]) for w in wins]) / np.mean([abs(l[2]-l[1]) for l in losses])


entry_price = df.iloc[i]["close"]
df['sl'] = entry_price - 0.5 * df.iloc[i]["atr"]
df['tp'] = entry_price + rr * (entry_price - sl)

print(f"Total trades: {len(results)}")
print(f"Win rate: {win_rate:.2%}")
print(f"Risk-Reward Ratio: {r_r_ratio:.2f}")
print(results)


Total trades: 585
Win rate: 30.26%
Risk-Reward Ratio: 2.26
[('loss', np.float64(1.17024), np.float64(1.17023)), ('loss', np.float64(1.17049), np.float64(1.1704050000000001)), ('win', np.float64(1.17052), np.float64(1.17081)), ('loss', np.float64(1.17062), np.float64(1.170585)), ('loss', np.float64(1.17063), np.float64(1.170595)), ('loss', np.float64(1.1696), np.float64(1.169365)), ('loss', np.float64(1.16961), np.float64(1.1695)), ('win', np.float64(1.16914), np.float64(1.1693500000000001)), ('loss', np.float64(1.16933), np.float64(1.1691799999999999)), ('loss', np.float64(1.16936), np.float64(1.169165)), ('loss', np.float64(1.1697), np.float64(1.16949)), ('win', np.float64(1.16916), np.float64(1.17015)), ('loss', np.float64(1.1648), np.float64(1.1645450000000002)), ('loss', np.float64(1.16431), np.float64(1.164125)), ('win', np.float64(1.16326), np.float64(1.16398)), ('win', np.float64(1.16334), np.float64(1.1639099999999998)), ('win', np.float64(1.164), np.float64(1.1642699999999997)

In [ ]:
prev = df.iloc[i-1]
def entring_stoploss_profit(row) :
    """hour = row['time'].hour
    idx = row.name
    if idx == 0:
        return None
    prev = df.iloc[idx - 1]"""

    if (
        row["close"] < row["vwap"] - 2 * row["atr"]
        and row["spread"] < 0.0002
        and row["volume_spike"]
        and row["close"] > row["open"]
        and prev["close"] < prev["open"]
        #and (hour < 11 or hour > 15)
    ):
        return "buy"
    return None
df["signal"] = df.apply(entring_stoploss_profit, axis=1)
df[["signal","time"]].dropna()


,signal,time
1164,buy,2025-09-01 19:24:00
1195,buy,2025-09-01 19:55:00
1196,buy,2025-09-01 19:56:00
1199,buy,2025-09-01 19:59:00
1230,buy,2025-09-01 20:30:00
...,...,...
71261,buy,2025-11-07 12:52:00
71283,buy,2025-11-07 13:14:00
71284,buy,2025-11-07 13:15:00
71299,buy,2025-11-07 13:30:00


In [ ]:
row = df[df["time"] == "2025-11-03 19:15:00"]
row

,time,open,high,low,close,tick_volume,spread,real_volume,atr,vwap,volume_spike,sl,tp,signal
65888,2025-11-03 19:15:00,1.15319,1.15342,1.15318,1.15341,48,0,0,0.00058,1.168358,True,1.155485,1.157305,buy


In [ ]:
"""rr = 3
for i in range(len(df["signal"])):
    entry_price = df.iloc[i]["close"]
    df["sl"] = entry_price - 0.5 * df.iloc[i]["atr"]
    df["tp"] = entry_price + rr * (entry_price - df["sl"])"""

'rr = 3\nfor i in range(len(df["signal"])):\n    entry_price = df.iloc[i]["close"]\n    df["sl"] = entry_price - 0.5 * df.iloc[i]["atr"]\n    df["tp"] = entry_price + rr * (entry_price - df["sl"])'

In [ ]:

# create trade logic
def on_bar(data, trades, orders):
    open_trades = trades[trades['state'] == 'open']
    num_open_trades = open_trades.shape[0]
    account_balance = 10000  # your account in USD
    risk_percent = 0.20  # 2%
    pip_value_per_lot = 10  # on EURUSD, 1 lot = $10 per pip
    risk_amount = account_balance * risk_percent
    # entry signal
    if data['signal'] == 'buy' :#and not num_open_trades:
        volume = 500000 #risk_amount / (stop_loss_pips * pip_value_per_lot)
        orders.open_trade(symbol, volume, 'buy')
    
    elif data['signal'] == 'sell' and not num_open_trades:
        volume = 500000 #risk_amount / (stop_loss_pips * pip_value_per_lot)
        orders.open_trade(symbol, volume, 'sell')
        
# exit signal
    if num_open_trades:
        trade = open_trades.iloc[0]

        if trade['order_type'] == 'buy' and data["low"] <= data["sl"] or data["high"] >= data["tp"]: 
            orders.close_trade(trade)
        elif trade['order_type'] == 'sell'  :
            orders.close_trade(trade)

In [ ]:
# backtest parameters
starting_balance = 10000
currency = 'USD'
exchange_rate = 1
commission = -7 / 500000

# backtest
bt = Backtester()
bt.set_starting_balance(starting_balance, currency=currency)
bt.set_exchange_rate(exchange_rate)
bt.set_commission(commission)

bt.set_historical_data(df)
bt.set_on_bar(on_bar)
#get_ohlc_history(symbol, timeframe, start_datetime, end_datetime)
bt.run_backtest()

bt.trades.tail(218)


,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,info,profit,commission,profit_net,profit_cumulative,balance
1185,closed,EURUSD,buy,500000,2025-11-03 19:15:00,1.15319,2025-11-03 19:16:00,1.1534,0,0,{},105.0,-7.0,98.0,80573.0,90573.0
1186,closed,EURUSD,buy,500000,2025-11-03 19:16:00,1.1534,2025-11-03 19:17:00,1.15363,0,0,{},115.0,-7.0,108.0,80681.0,90681.0
1187,closed,EURUSD,buy,500000,2025-11-03 19:43:00,1.15316,2025-11-03 19:44:00,1.15322,0,0,{},30.0,-7.0,23.0,80704.0,90704.0
1188,closed,EURUSD,buy,500000,2025-11-03 20:33:00,1.15236,2025-11-03 20:34:00,1.15239,0,0,{},15.0,-7.0,8.0,80712.0,90712.0
1189,closed,EURUSD,buy,500000,2025-11-03 20:36:00,1.15218,2025-11-03 20:37:00,1.15234,0,0,{},80.0,-7.0,73.0,80785.0,90785.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1398,closed,EURUSD,buy,500000,2025-11-07 12:52:00,1.15491,2025-11-07 12:53:00,1.15501,0,0,{},50.0,-7.0,43.0,88167.0,98167.0
1399,closed,EURUSD,buy,500000,2025-11-07 13:14:00,1.15501,2025-11-07 13:15:00,1.1551,0,0,{},45.0,-7.0,38.0,88205.0,98205.0
1400,closed,EURUSD,buy,500000,2025-11-07 13:15:00,1.1551,2025-11-07 13:16:00,1.15515,0,0,{},25.0,-7.0,18.0,88223.0,98223.0
1401,closed,EURUSD,buy,500000,2025-11-07 13:30:00,1.15577,2025-11-07 14:57:00,1.15556,0,0,{},-105.0,-7.0,-112.0,88111.0,98111.0


In [ ]:
mm = bt.plot_balance()
display(mm)

In [ ]:
evaluate_backtest(bt.trades)

Daily Drawdown : -890.00
Max Drawdown (portfolio): -960.00
worst trades (trade MAE): [np.float64(0.0), np.float64(-9.99999999995449), np.float64(-5.000000000032756), np.float64(-4.999999999921734), np.float64(-5.000000000032756), np.float64(0.0), np.float64(-14.999999999987246), np.float64(0.0), np.float64(-9.99999999995449), np.float64(-5.000000000032756), np.float64(0.0), np.float64(-5.000000000032756), np.float64(0.0), np.float64(0.0), np.float64(-44.99999999996174), np.float64(0.0), np.float64(-70.0000000000145), np.float64(-20.000000000020002), np.float64(-4.999999999921734), np.float64(-10.000000000065512), np.float64(0.0), np.float64(-35.00000000000725), np.float64(-49.99999999999449), np.float64(-15.000000000098268), np.float64(-5.000000000032756), np.float64(-5.000000000032756), np.float64(-5.000000000032756), np.float64(-44.99999999996174), np.float64(-90.0000000000345), np.float64(-9.99999999995449), np.float64(-14.999999999987246), np.float64(-35.00000000000725), np.float64

,order_type,profit
0,buy,97750.0


,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,...,profit,commission,profit_net,profit_cumulative,balance,date,time,intrabar_drawdown,current_max,drawdown1
0,closed,EURUSD,buy,500000,2025-09-01 19:24:00,1.17005,2025-09-01 19:25:00,1.17024,0,0,...,95.0,-7.0,88.0,88.0,10088.0,2025-09-01,2025-09-01 19:25:00,0.0,88.0,0.0
1,closed,EURUSD,buy,500000,2025-09-01 19:55:00,1.17037,2025-09-01 19:56:00,1.17039,0,0,...,10.0,-7.0,3.0,91.0,10091.0,2025-09-01,2025-09-01 19:56:00,-10.0,91.0,0.0
2,closed,EURUSD,buy,500000,2025-09-01 19:56:00,1.17039,2025-09-01 19:57:00,1.17041,0,0,...,10.0,-7.0,3.0,94.0,10094.0,2025-09-01,2025-09-01 19:57:00,-5.0,94.0,0.0
3,closed,EURUSD,buy,500000,2025-09-01 19:59:00,1.1704,2025-09-01 20:00:00,1.17049,0,0,...,45.0,-7.0,38.0,132.0,10132.0,2025-09-01,2025-09-01 20:00:00,-5.0,132.0,0.0
4,closed,EURUSD,buy,500000,2025-09-01 20:30:00,1.17026,2025-09-01 20:31:00,1.17031,0,0,...,25.0,-7.0,18.0,150.0,10150.0,2025-09-01,2025-09-01 20:31:00,-5.0,150.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1396,closed,EURUSD,buy,500000,2025-11-07 09:27:00,1.15346,2025-11-07 09:28:00,1.15348,0,0,...,10.0,-7.0,3.0,88116.0,98116.0,2025-11-07,2025-11-07 09:28:00,-25.0,88116.0,0.0
1397,closed,EURUSD,buy,500000,2025-11-07 09:42:00,1.15322,2025-11-07 09:43:00,1.15325,0,0,...,15.0,-7.0,8.0,88124.0,98124.0,2025-11-07,2025-11-07 09:43:00,-45.0,88124.0,0.0
1398,closed,EURUSD,buy,500000,2025-11-07 12:52:00,1.15491,2025-11-07 12:53:00,1.15501,0,0,...,50.0,-7.0,43.0,88167.0,98167.0,2025-11-07,2025-11-07 12:53:00,0.0,88167.0,0.0
1399,closed,EURUSD,buy,500000,2025-11-07 13:14:00,1.15501,2025-11-07 13:15:00,1.1551,0,0,...,45.0,-7.0,38.0,88205.0,98205.0,2025-11-07,2025-11-07 13:15:00,-30.0,88205.0,0.0


,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,...,profit,commission,profit_net,profit_cumulative,balance,date,time,intrabar_drawdown,current_max,drawdown1
202,closed,EURUSD,buy,500000,2025-09-29 22:59:00,1.17275,2025-09-29 23:00:00,1.17274,0,0,...,-5.0,-7.0,-12.0,8049.0,18049.0,2025-09-29,2025-09-29 23:00:00,-45.0,8061.0,-12.0
345,closed,EURUSD,buy,500000,2025-10-08 19:33:00,1.15996,2025-10-08 19:34:00,1.15995,0,0,...,-5.0,-7.0,-12.0,15243.0,25243.0,2025-10-08,2025-10-08 19:34:00,-50.0,15255.0,-12.0
379,closed,EURUSD,buy,500000,2025-10-09 19:00:00,1.15591,2025-10-09 19:11:00,1.15555,0,0,...,-180.0,-7.0,-187.0,17370.0,27370.0,2025-10-09,2025-10-09 19:11:00,-250.0,17557.0,-187.0
382,closed,EURUSD,buy,500000,2025-10-09 23:16:00,1.15644,2025-10-10 00:00:00,1.15642,0,0,...,-10.0,-7.0,-17.0,17449.0,27449.0,2025-10-09,2025-10-10 00:00:00,-575.0,17557.0,-108.0
383,closed,EURUSD,buy,500000,2025-10-09 23:50:00,1.15648,2025-10-10 00:01:00,1.15537,0,0,...,-555.0,-7.0,-562.0,16887.0,26887.0,2025-10-09,2025-10-10 00:01:00,-595.0,17557.0,-670.0
407,closed,EURUSD,buy,500000,2025-10-10 06:56:00,1.15729,2025-10-10 07:57:00,1.15725,0,0,...,-20.0,-7.0,-27.0,26459.0,36459.0,2025-10-10,2025-10-10 07:57:00,-270.0,26493.0,-34.0
411,closed,EURUSD,buy,500000,2025-10-10 08:00:00,1.15736,2025-10-10 08:04:00,1.1572,0,0,...,-80.0,-7.0,-87.0,26821.0,36821.0,2025-10-10,2025-10-10 08:04:00,-90.0,26908.0,-87.0
488,closed,EURUSD,buy,500000,2025-10-14 05:55:00,1.1575,2025-10-14 06:02:00,1.15738,0,0,...,-60.0,-7.0,-67.0,38222.0,48222.0,2025-10-14,2025-10-14 06:02:00,-100.0,38289.0,-67.0
505,closed,EURUSD,buy,500000,2025-10-14 15:15:00,1.15592,2025-10-14 15:38:00,1.15554,0,0,...,-190.0,-7.0,-197.0,38583.0,48583.0,2025-10-14,2025-10-14 15:38:00,-225.0,38780.0,-197.0
506,closed,EURUSD,buy,500000,2025-10-14 15:16:00,1.15601,2025-10-14 15:39:00,1.1555,0,0,...,-255.0,-7.0,-262.0,38321.0,48321.0,2025-10-14,2025-10-14 15:39:00,-325.0,38780.0,-459.0


,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,...,date,time,intrabar_drawdown,current_max,drawdown1,dayofweek,hourofday,month,year,drawdown
0,closed,EURUSD,buy,500000,2025-09-01 19:24:00,1.17005,2025-09-01 19:25:00,1.17024,0,0,...,2025-09-01,2025-09-01 19:25:00,0.0,88.0,0.0,0,19,2025-09,2025,0.0
1,closed,EURUSD,buy,500000,2025-09-01 19:55:00,1.17037,2025-09-01 19:56:00,1.17039,0,0,...,2025-09-01,2025-09-01 19:56:00,-10.0,91.0,0.0,0,19,2025-09,2025,0.0
2,closed,EURUSD,buy,500000,2025-09-01 19:56:00,1.17039,2025-09-01 19:57:00,1.17041,0,0,...,2025-09-01,2025-09-01 19:57:00,-5.0,94.0,0.0,0,19,2025-09,2025,0.0
3,closed,EURUSD,buy,500000,2025-09-01 19:59:00,1.1704,2025-09-01 20:00:00,1.17049,0,0,...,2025-09-01,2025-09-01 20:00:00,-5.0,132.0,0.0,0,19,2025-09,2025,0.0
4,closed,EURUSD,buy,500000,2025-09-01 20:30:00,1.17026,2025-09-01 20:31:00,1.17031,0,0,...,2025-09-01,2025-09-01 20:31:00,-5.0,150.0,0.0,0,20,2025-09,2025,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1398,closed,EURUSD,buy,500000,2025-11-07 12:52:00,1.15491,2025-11-07 12:53:00,1.15501,0,0,...,2025-11-07,2025-11-07 12:53:00,0.0,88167.0,0.0,4,12,2025-11,2025,0.0
1399,closed,EURUSD,buy,500000,2025-11-07 13:14:00,1.15501,2025-11-07 13:15:00,1.1551,0,0,...,2025-11-07,2025-11-07 13:15:00,-30.0,88205.0,0.0,4,13,2025-11,2025,0.0
1400,closed,EURUSD,buy,500000,2025-11-07 13:15:00,1.1551,2025-11-07 13:16:00,1.15515,0,0,...,2025-11-07,2025-11-07 13:16:00,-45.0,88223.0,0.0,4,13,2025-11,2025,0.0
1401,closed,EURUSD,buy,500000,2025-11-07 13:30:00,1.15577,2025-11-07 14:57:00,1.15556,0,0,...,2025-11-07,2025-11-07 14:57:00,-180.0,88223.0,-112.0,4,13,2025-11,2025,-112.0


max_drawdown -670.0
